In [8]:
import pandas as pandas
import os

from dotenv import load_dotenv
from src.data.blob_utils import get_list_of_files
from azure.storage.blob import BlobServiceClient

ModuleNotFoundError: No module named 'src'

In [6]:
load_dotenv()

True

In [7]:
service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_CONNECTION_STRING"))
container_client = service_client.get_container_client(os.getenv("CONTAINER_NAME"))

NameError: name 'BlobServiceClient' is not defined

In [ ]:
files = get_list_of_files(container_client)